In [1]:
using LowRankModels
using CSV
using DataFrames
using Pkg
using StatsBase
using Random, SparseArrays

using Random
using LinearAlgebra
using DataFrames             # Data tables are called "DataFrames"
using StatsPlots             # load plotting packages 
using Statistics             # basic statistical functions
using CSV  
using SparseArrays
using Pkg
using StatsBase
pyplot()

Plots.PyPlotBackend()

In [2]:
df2 = CSV.read("../data/NYC_SPARCS_cancer.csv")
df = CSV.read("../data/SPARCS_cancer_binary.csv")



,Column1,Total_Charges,APR_Severity_of_Illness_Code,APR_Risk_of_Mortality,Age_Ordinal
,Int64,Float64,Int64,Int64,Int64
1,0,216494.0,3,4,3
2,1,194331.0,4,3,5
3,2,79396.5,3,2,4
4,3,86736.8,3,4,4
5,4,178593.0,3,1,1
6,5,57239.2,2,4,4
7,6,111640.0,4,2,4
8,7,31801.9,2,4,4
9,8,38714.5,3,2,5


In [3]:
for name in names(df)
    println(":", name, ", ")
end

:Column1, 
:Total_Charges, 
:APR_Severity_of_Illness_Code, 
:APR_Risk_of_Mortality, 
:Age_Ordinal, 
:Length_of_Stay, 
:Hospital.County.Bronx, 
:Hospital.County.Kings, 
:Hospital.County.Manhattan, 
:Hospital.County.Queens, 
:Hospital.County.Richmond, 
:Type.of.Admission.Elective, 
:Type.of.Admission.Emergency, 
:Type.of.Admission.Newborn, 
:Type.of.Admission.Urgent, 
:Type.of.Admission.Not Available, 
:Type.of.Admission.Trauma, 
:Patient.Disposition.Home w/ Home Health Services, 
:Patient.Disposition.Skilled Nursing Home, 
:Patient.Disposition.Home or Self Care, 
:Patient.Disposition.Expired, 
:Patient.Disposition.Left Against Medical Advice, 
:Patient.Disposition.Short-term Hospital, 
:Patient.Disposition.Hospice - Home, 
:Patient.Disposition.Inpatient Rehabilitation Facility, 
:Patient.Disposition.Medicare Cert Long Term Care Hospital, 
:Patient.Disposition.Another Type Not Listed, 
:Patient.Disposition.Hospice - Medical Facility, 
:Patient.Disposition.Cancer Center or Children's Hosp

:Payment.Typology.3.Federal/State/Local/VA, 
:Payment.Typology.3.Unknown, 
:Payment.Typology.3.Miscellaneous/Other, 
:Payment.Typology.3.Managed Care, Unspecified, 
:Emergency.Department.Indicator.Y, 
:Gender_F, 
:Race_Black/African_American, 
:Race_Other_Race, 
:Race_White, 
:Race_Unknown, 
:Ethnicity_Not_Span/Hispanic, 
:Ethnicity_Spanish/Hispanic, 
:Ethnicity.Unknown, 


In [4]:
features = [
    :Total_Charges,
    :APR_Severity_of_Illness_Code,
    :APR_Risk_of_Mortality,
    :Age_Ordinal,
    :Length_of_Stay, 
    :Gender_F
]

#df = df[:,[1 2]]
#df = select!(df, Not(:B))

6-element Array{Symbol,1}:
 :Total_Charges
 :APR_Severity_of_Illness_Code
 :APR_Risk_of_Mortality
 :Age_Ordinal
 :Length_of_Stay
 :Gender_F

In [5]:
dfsmall1 = df[features]

┌ Warning: `getindex(df::DataFrame, col_inds::Union{AbstractVector, Regex, Not})` is deprecated, use `df[:, col_inds]` instead.
│   caller = top-level scope at In[5]:1
└ @ Core In[5]:1


,Total_Charges,APR_Severity_of_Illness_Code,APR_Risk_of_Mortality,Age_Ordinal
,Float64,Int64,Int64,Int64
1,216494.0,3,4,3
2,194331.0,4,3,5
3,79396.5,3,2,4
4,86736.8,3,4,4
5,178593.0,3,1,1
6,57239.2,2,4,4
7,111640.0,4,2,4
8,31801.9,2,4,4
9,38714.5,3,2,5


# kmeans k=4 quadloss

In [6]:
k = 4
A= dfsmall1
losses = QuadLoss() # minimize squared distance to cluster centroids
rx = UnitOneSparseConstraint() # each row is assigned to exactly one cluster
ry = ZeroReg() # no regularization on the cluster centroids
glrm = GLRM(A,losses,rx,ry,k)

GLRM(35804×6 DataFrame. Omitted printing of 3 columns
│ Row   │ Total_Charges │ APR_Severity_of_Illness_Code │ APR_Risk_of_Mortality │
│       │ Float64       │ Int64                        │ Int64                 │
├───────┼───────────────┼──────────────────────────────┼───────────────────────┤
│ 1     │ 216494.0      │ 3                            │ 4                     │
│ 2     │ 194331.0      │ 4                            │ 3                     │
│ 3     │ 79396.5       │ 3                            │ 2                     │
│ 4     │ 86736.8       │ 3                            │ 4                     │
│ 5     │ 178593.0      │ 3                            │ 1                     │
│ 6     │ 57239.2       │ 2                            │ 4                     │
│ 7     │ 111640.0      │ 4                            │ 2                     │
│ 8     │ 31801.9       │ 2                            │ 4                     │
│ 9     │ 38714.5       │ 3                            

In [7]:
X,Y,ch = fit!(glrm)

Fitting GLRM
Iteration 10: objective value = 1.1260116404158503e14
Iteration 20: objective value = 1.0367420177870258e14
Iteration 30: objective value = 1.0242169766939364e14
Iteration 40: objective value = 1.0182047039115008e14
Iteration 50: objective value = 1.0155088972164355e14
Iteration 60: objective value = 1.0138969526868458e14


([0.0 0.0 … 0.0 0.0; 1.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0 1.0], [53480.12123852681 2.1720924583606105 … 6.4583527378706425 0.5445390226823665; 289520.8382158069 3.3874345866904645 … 32.752824159924884 0.44313309956043306; 67824.6511289402 2.353207581365324 … 8.522566901452839 0.5448125360232189; 43908.948829005574 2.053828555535085 … 5.535646453511289 0.5308618110365382], ConvergenceHistory("ProxGradGLRM", [Inf, 3.308709772347356e14, 1.4105459168234512e14, 1.3315497983254047e14, 1.3230819844117519e14, 1.3124786564012808e14, 1.2895113534805642e14, 1.2428120724058702e14, 1.191819944980284e14, 1.1550405494356056e14  …  1.015171484622014e14, 1.0149333155655828e14, 1.0147200526109481e14, 1.0145955998936077e14, 1.0144455487172311e14, 1.0143372086620772e14, 1.0141538365645012e14, 1.0140278861067028e14, 1.0138969526868458e14, 1.0138038037057972e14], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0,

In [ ]:
println("Computing cross validation error for each fold")
params = Params(1.0, max_iter=100, abs_tol=0.0, min_stepsize=.001)
train_error, test_error, train_glrms, test_glrms = cross_validate(glrm, nfolds=4, params=params)
df = DataFrame(train_error = train_error, test_error = test_error)

In [ ]:
X = convert(DataFrame, X)
#CSV.write("../data/cancer4quad.csv", X)

In [ ]:
CSV.write("../data/cancer4quad.csv", X)

# kmeans k=4 different loss init_kmeans

In [ ]:
k = 4
A= dfsmall1
losses = QuadLoss() # minimize squared distance to cluster centroids
rx = UnitOneSparseConstraint() # each row is assigned to exactly one cluster
ry = ZeroReg() # no regularization on the cluster centroids
glrm = GLRM(A,losses,rx,ry,k)

# function kmeans_proxgrad_drg(kval, lossarr, matrix)
    k = kval
    losses = lossarr
    A = matrix
    rx = UnitOneSparseConstraint()
    ry = ZeroReg()
    glrm = GLRM(A,losses,rx,ry,k)
    params = ProxGradParams(inner_iter_Y=40)

    
    X,Y,ch = fit!(glrm, params) # objective value
    println("X: ")
    println(X)
    println("Y: ")
    println(Y)
    
    imputey = impute(losses, Y) 
    imputey  = convert(DataFrame, imputey) 
    CSV.write(string("../data/iyproxgrad_drg", k,".csv" ), imputey)
    #CSV.read(string("../data/iywp", k,".csv" ), imputey)
    
    X= convert(DataFrame, X) 
    CSV.write(string("../data/Xproxgrad_drg", k,".csv" ), X)  
    #CSV.read(string("../data/Xwp", k,".csv" ), X)
#end   

In [ ]:
loss1 = fill(HuberLoss(),1)
loss2 = fill(OrdinalHingeLoss(), 4)
loss3 = fill(HingeLoss(), 1)

lossarr = [loss1; loss2; loss3]
lossarr


In [ ]:
#init kmean demo
init_kmeanspp!(glrm)
X,Y,ch = fit!(glrm)